In [1]:
#import
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization, Dropout
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [2]:
#data preparing
def convert_data_csv_to_numpy(data, sorting = False): # pandas 통해 읽은 csv data numpy 형태로 변경
  if sorting == True:
    data = data.sort_values(by=[0], axis=0)

  label = np.array(data[0]) # csv file 에서 0번째 colum은 index임
  only_data = np.array(data.drop([0], axis = 1)) # csv file에서 0번 째 colum 탈락 -> data만 남게 됨
  # only_data = tf.convert_to_tensor(only_data, dtype=tf.float32)
  only_data = tf.keras.utils.normalize(only_data, axis=-1, order=2).reshape((-1, 28, 28, 1)) #data nomalize & reshaping
  return only_data, label

In [3]:
#load
train_df = pd.read_csv("./emnist-byclass-train.csv", header=None)
test_df = pd.read_csv("./emnist-byclass-test.csv", header=None)

In [4]:
#change to numpy
X_train, y_train  = convert_data_csv_to_numpy(train_df)
X_test, y_test  = convert_data_csv_to_numpy(test_df)

train_df = None
test_df = None

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [7]:
print(X_train.shape, X_val.shape, X_test.shape)

(558345, 28, 28, 1) (139587, 28, 28, 1) (116323, 28, 28, 1)


In [8]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

In [9]:
print(y_train.shape,y_val.shape, y_test.shape)

(558345, 62) (139587, 62) (116323, 62)


In [10]:
X_train = tf.convert_to_tensor(X_train.copy(), dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val.copy(), dtype=tf.float32)

In [12]:
print(type(X_train))
print(type(X_val))
print(X_train.shape)
print(X_val.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
(558345, 28, 28, 1)
(139587, 28, 28, 1)


In [13]:
f_train_x = tf.image.grayscale_to_rgb(X_train)
f_val_x = tf.image.grayscale_to_rgb(X_val)

In [14]:
#final data split
boundary = int(X_train.shape[0]*0.8)

final_train = f_train_x[:boundary]
final_train_y = y_train[:boundary]

final_val = f_train_x[boundary:]
final_val_y = y_train[boundary:]

In [21]:
final_train[0].shape

TensorShape([28, 28, 3])

In [22]:
mean = np.mean(X_train)
std = np.std(X_train)

In [23]:
print(mean)
print(std)

0.016407458
0.031722296


In [24]:
#efficientent _model0
IMG_SHAPE = (224, 224, 3)
model0 = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [25]:
#make new model _eff_model
tf.keras.utils.plot_model(model0) # to draw and visualize

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(62, activation = 'softmax')

eff_model = keras.models.Sequential()
eff_model.add(tf.keras.layers.Normalization( axis=-1, mean=0.016425587, variance=0.03171295, input_shape = (28, 28, 3)))
eff_model.add(tf.keras.layers.Resizing(height = 224, width = 224))
eff_model.add(model0)
eff_model.add(global_average_layer)
eff_model.add(tf.keras.layers.Dense(1000))
eff_model.add(tf.keras.layers.ReLU())
eff_model.add(prediction_layer)


eff_model.summary() # to see the list of layers and parameters


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 28, 28, 3)        0         
 ion)                                                            
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)          

In [42]:
print(final_train.shape)
print(y_train.shape)
print(final_train_y.shape)
print(final_val.shape)
print(y_val.shape)
print(final_val_y.shape)

(446676, 28, 28, 3)
(558345, 62)
(446676, 62)
(111669, 28, 28, 3)
(139587, 62)
(111669, 62)


In [27]:
eff_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [28]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

eff_model_history = eff_model.fit(final_train, final_train_y, epochs=10, validation_data=(final_val, final_val_y), callbacks=[early_stopping_cb])

Epoch 1/10


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[32,112,112,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node sequential/efficientnetb0/block1a_activation/mul_1-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_19673]